<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# EPAT Session 2

**Executive Program in Algorithmic Trading**

**_OOP & Event-Based Backtesting_**

Prof. Dr. Yves J. Hilpisch | The Python Quants GmbH | http://tpq.io

<a href="https://home.tpq.io/certificates/pyalgo" target="_blank"><img src="https://hilpisch.com/pyalgo_cover_color.png" width="300px" align="left" border="1px"></a>

## Basic Imports

In [ ]:
import time
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
pd.set_option('mode.chained_assignment', None)

## OOP

In [ ]:
pd.__version__  # package

In [ ]:
pd.DataFrame  # class from the package

In [ ]:
df = pd.DataFrame()  # instance of the class

In [ ]:
type(df)  # type of object

## Financial Data Class

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'  # EOD data
# url = 'http://hilpisch.com/aiif_eikon_id_data.csv'  # intraday data

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
# raw.info()

In [ ]:
class FinancialData:
    url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'
    def __init__(self, symbol):
        self.symbol = symbol
        self.retrieve_data()
        self.prepare_data()
    def retrieve_data(self):
        self.raw = pd.read_csv(self.url, index_col=0, parse_dates=True).dropna()
    def prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol])
        self.data['returns'] = np.log(self.data[self.symbol] / self.data[self.symbol].shift(1))
    def plot_data(self, cols=None):
        if cols is None:
            cols = [self.symbol]
        self.data[cols].plot(figsize=(10, 6));

In [ ]:
fd = FinancialData('.SPX')

In [ ]:
fd.data.info()

In [ ]:
fd.plot_data()

## Event-Based View/Approach

In [ ]:
for bar in range(10):
    print(bar, raw.index[bar], raw['EUR='].iloc[bar])
    time.sleep(1)

## Event-Based Backtesting (Base Class)

In [ ]:
class BacktestingBase(FinancialData):
    def __init__(self, symbol, amount):
        super(BacktestingBase, self).__init__(symbol)
        self.current_balance = amount
        self.initial_balance = amount
        self.position = 0
        self.trades = 0
        self.units = 0
    def get_date_price(self, bar):
        date = str(self.raw.index[bar])[:10]
        price = self.raw[self.symbol].iloc[bar]
        return date, price
    def get_current_balance(self, bar):
        date, price = self.get_date_price(bar)
        print(f'{date} | current balance = {self.current_balance}')
    def get_net_wealth(self, bar):
        date, price = self.get_date_price(bar)
        net_wealth = self.current_balance + self.units * price 
        print(f'{date} | current net wealth = {net_wealth}')
    def place_buy_order(self, bar, amount=None, units=None):
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / price)
        self.current_balance -= units * price
        self.units += units
        self.trades += 1
        print(f'{date} | buying {units} units at {price}')
    def place_sell_order(self, bar, amount=None, units=None):
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / price)
        self.current_balance += units * price
        self.units -= units
        self.trades += 1
        print(f'{date} | selling {units} units at {price}')
    def close_out(self, bar):
        date, price = self.get_date_price(bar)
        self.current_balance += self.units * price
        self.trades +=1
        perf = (self.current_balance - self.initial_balance) / self.initial_balance * 100
        print(55 * '=')
        print(f'{date} | *** CLOSING OUT ***')
        print(f'{date} | closing {self.units} at {price}')
        self.units = 0
        self.position = 0
        self.get_current_balance(bar)
        self.get_net_wealth(bar)
        print(f'{date} | performance [%] = {perf:.3f}')
        print(f'{date} | trades [#] = {self.trades}')

In [ ]:
bb = BacktestingBase('MSFT.O', 10000)

In [ ]:
bb.data.info()

In [ ]:
bb.get_date_price(100)

In [ ]:
bb.get_current_balance(150)

In [ ]:
bb.place_buy_order(150, units=100)

In [ ]:
bb.get_current_balance(150)

In [ ]:
bb.units

In [ ]:
bb.get_net_wealth(150)

In [ ]:
bb.get_net_wealth(250)

In [ ]:
bb.place_sell_order(700, units=50)

In [ ]:
bb.get_current_balance(700)

In [ ]:
bb.get_net_wealth(700)

In [ ]:
bb.units

In [ ]:
bb.close_out(1000)

## Event-Based Backtesting (SMA Strategy)

In [ ]:
class SMABacktester(BacktestingBase):
    def prepare_statistics(self):
        self.data['SMA1'] = self.data[self.symbol].rolling(self.SMA1).mean()
        self.data['SMA2'] = self.data[self.symbol].rolling(self.SMA2).mean()
    
    def backtest_strategy(self, SMA1, SMA2):
        self.SMA1 = SMA1
        self.SMA2 = SMA2
        self.prepare_statistics()
        self.units = 0
        self.position = 0
        self.trades = 0
        self.current_balance = self.initial_balance
        self.data['signal'] = np.where(self.data['SMA1'] > self.data['SMA2'], 1, -1)
        for bar in range(self.SMA2, len(self.data)):
            date, price = self.get_date_price(bar)
            if self.position in [0, -1] and self.data['signal'].iloc[bar] == 1:
                print(55 * '=')
                print(f'{date} | *** GOING LONG ***')
                self.place_buy_order(bar, units=(1 - self.position) * 7250)
                self.position = 1
                self.get_current_balance(bar)
                self.get_net_wealth(bar)
            elif self.position in [0, 1] and self.data['signal'].iloc[bar] == -1:
                print(55 * '=')
                print(f'{date} | *** GOING SHORT ***')
                self.place_sell_order(bar, units=(1 + self.position) * 7250)
                self.position = -1
                self.get_current_balance(bar)
                self.get_net_wealth(bar)
        self.close_out(bar)

In [ ]:
sma = SMABacktester('EUR=', 10000)

In [ ]:
sma.backtest_strategy(42, 252)

<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>